# Data Preprocessing and Analysis

This notebook preprocesses the dataset by loading, cleaning, normalizing. We focus on calculating minimum, maximum, and range values.

---

## Step 1: Importing Libraries

We use `pandas` for data manipulation, `matplotlib` for plotting, and `statsmodels` for time series analysis.


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose


## Step 2: Load and Inspect Data

Load the dataset and inspect its structure. The first column is the "Page" identifier, and the remaining columns are date values.


In [ ]:
# Load the CSV file
file_path = "./Data/train_1.csv"
df = pd.read_csv(file_path)

# Display the first few rows and summary info
print(df.head())
print(df.info())
print(df.describe())


                                                Page  2015-07-01  2015-07-02  \
0            2NE1_zh.wikipedia.org_all-access_spider        18.0        11.0   
1             2PM_zh.wikipedia.org_all-access_spider        11.0        14.0   
2              3C_zh.wikipedia.org_all-access_spider         1.0         0.0   
3         4minute_zh.wikipedia.org_all-access_spider        35.0        13.0   
4  52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...         NaN         NaN   

   2015-07-03  2015-07-04  2015-07-05  2015-07-06  2015-07-07  2015-07-08  \
0         5.0        13.0        14.0         9.0         9.0        22.0   
1        15.0        18.0        11.0        13.0        22.0        11.0   
2         1.0         1.0         0.0         4.0         0.0         3.0   
3        10.0        94.0         4.0        26.0        14.0         9.0   
4         NaN         NaN         NaN         NaN         NaN         NaN   

   2015-07-09  ...  2016-12-22  2016-12-23  2016-12-24  

## Step 3: Cleaning Data

Clean the dataset field where if the specific page blank threshold is more than 1 month

In [7]:
missing_threshold = 0.05
date_columns = df.columns[1:]  # to avoid first column 'Page'

df['missing_percentage'] = df[date_columns].isnull().mean(axis=1)

# Filter out
df_cleaned = df[df['missing_percentage'] < missing_threshold]

# Drop 'missing_percentage'
df_cleaned = df_cleaned.drop(columns=['missing_percentage'])

output_file_path = "cleaned_data.csv"
df_cleaned.to_csv(output_file_path, index=False)

print(f"Cleaned data saved to {output_file_path}")


Cleaned data saved to cleaned_data.csv


## Step 4: Filling Data

Populating the dataset fields where blank data is present with the average of page visits for that month

In [ ]:
# Load the CSV file
file_path = "cleaned_data.csv"
df = pd.read_csv(file_path)

# Convert date columns to datetime format
df.columns = ['Page'] + pd.to_datetime(df.columns[1:]).strftime('%Y-%m-%d').tolist()

# Set 'Page' column as index temporarily for easier handling
df.set_index('Page', inplace=True)

# Transpose the DataFrame to work with dates as the index
df_transposed = df.transpose()
df_transposed.index = pd.to_datetime(df_transposed.index)

# Resample data to monthly frequency and calculate the mean for each page, then round to integers
monthly_avg = df_transposed.resample('ME').mean().round().astype(int)

# Backfill missing data using the monthly average for each page
for date, row in df_transposed.iterrows():
    month = date.strftime('%Y-%m')
    # Ensure we only use the Series for the correct month
    if month in monthly_avg.index.strftime('%Y-%m'):
        # Get the monthly average as a Series for that month (already converted to integers)
        month_avg_series = monthly_avg.loc[monthly_avg.index.strftime('%Y-%m') == month].iloc[0]
        # Fill missing values in this row with the integer monthly average for each page
        df_transposed.loc[date] = row.fillna(month_avg_series)

# Transpose back to original format
df_filled = df_transposed.transpose().reset_index()

# Save the updated DataFrame
output_file_path = "filled_data.csv"
df_filled.to_csv(output_file_path, index=False)

print(f"Filled data saved to {output_file_path}")


Filled data saved to filled_data.csv


## Step 5: Calculate Min, Max, and Range Values

For each page, calculate the minimum, maximum, and range of values to understand the spread.


In [ ]:
file_path = "./filled_data.csv"
df = pd.read_csv(file_path)

# Selecting only the numerical columns
numeric_cols = df.columns[1:]  # first column is 'Page'

# Transpose back temporarily to calculate row-based min, max, and range
df_temp = df[numeric_cols].T  # Transpose for calculation convenience
df_temp['Min_Value'] = df_temp.min(axis=1)
df_temp['Max_Value'] = df_temp.max(axis=1)
df_temp['Range'] = df_temp['Max_Value'] - df_temp['Min_Value']

# Display min, max, and range for each page
print(df_temp[['Min_Value', 'Max_Value', 'Range']].head())

# Transpose back to original for further time series analysis
df = df_temp.drop(columns=['Min_Value', 'Max_Value', 'Range']).T


                     Min_Value   Max_Value       Range
2015-07-01 00:00:00        0.0  20381245.0  20381245.0
2015-07-02 00:00:00        0.0  20752194.0  20752194.0
2015-07-03 00:00:00        0.0  19573967.0  19573967.0
2015-07-04 00:00:00        0.0  20439645.0  20439645.0
2015-07-05 00:00:00        0.0  20772109.0  20772109.0


## Step 6: Min-Max Normalizing data

Normalize values between 0 and 1 to standardize across different pages.

In [ ]:
# Selecting only the numerical columns
numeric_cols = df.columns[1:]  # first column 'Page'

# 1. Min-Max Normalization
df_min_max_norm = df.copy()
df_min_max_norm[numeric_cols] = (df[numeric_cols] - df[numeric_cols].min()) / (df[numeric_cols].max() - df[numeric_cols].min())
# Export to CSV
df_min_max_norm.to_csv("min_max_normalized_data.csv", index=False)

# Displaying the normalized data
# print("Data after Min-Max Normalization:")
# print(df_min_max_norm[['Page'] + list(numeric_cols)].head())

print("Normalized data has been saved to CSV files.")

Normalized data has been saved to CSV files.
